In [2]:
from math import log
from bitarray import bitarray 
from random import randint, choice
from collections import defaultdict
import string

%load_ext pycodestyle_magic

In [4]:
%%pycodestyle


class CustomBloom(object):

    def __init__(self, fpr, s):
        self.hash_size = int(-(s * log(fpr)) / (log(2)**2))
        self.hash_count = int((self.hash_size/s) * log(2))
        self.hash = bitarray(self.hash_size)
        self.hash.setall(False)
        self.p = randint(s, s + 1000000)
        self.x = randint(1, self.p - 1)

    def string_hash(self, string):
        h = 0
        for j in range(len(string)-1, -1, -1):
            h = (h * self.x + ord(string[j]) + self.p) % self.p
        return h

    def multi_hash(self, string):
        for x in range(self.hash_count):
            h = self.string_hash(string)
            yield h

    def insert(self, item):
        for key in self.multi_hash(item):
            self.hash[key] = True

    def lookup(self, item):
        for key in self.multi_hash(item):
            if self.hash[key] is False:
                return False
        return True


In [8]:
%%pycodestyle


def gen_lines(length, number):
    output = []
    for x in range(number):
        output.append(''.join([choice(characters) for x in range(length)]))
    return output


def test_bloomfilter(n_run, param, char_num, lines_num, lines, lines_test):
    result = defaultdict()
    fpr, s = param
    for run in range(n_run):
        bloom = CustomBloom(fpr, s)
        false = 0
        for line in lines:
            bloom.insert(line)
        for line in lines_test:
            if bloom.lookup(line) is True and line not in lines:
                false += 1
        score = 1 - (len(lines_test) - false) / len(lines_test)
        result[run + 1] = score
    return result


In [9]:
characters = string.ascii_letters + string.digits

In [10]:
lines = gen_lines(30, 10**4)
lines_test = gen_lines(30, 10**4)

In [11]:
%%time

result = test_bloomfilter(5, [0.05, 10**6], 20, 10**4 , lines, lines_test)

CPU times: user 5.42 s, sys: 24.1 ms, total: 5.44 s
Wall time: 5.46 s


In [14]:
print('For 10**4(10000) elements FPR results of our Bloom Filter:\n')
for el in result:
    print('For run n.{} FPR score is:\t{}'.format(el, result[el]))

For 10**4(10000) elements FPR results of our Bloom Filter:

For run n.1 FPR score is:	0.009000000000000008
For run n.2 FPR score is:	0.0048000000000000265
For run n.3 FPR score is:	0.006099999999999994
For run n.4 FPR score is:	0.009099999999999997
For run n.5 FPR score is:	0.005800000000000027
